# Start
```
筆電之路徑為 C:/Cody/Research/clean_data/prepare.csv
桌機之路徑為 D:/Research/clean_data/prepare.csv
```

In [4]:
select_mode = 1
# 0 for notebook ver
# 1 for desktop ver

if select_mode == 0:
    file_path = r"C:/Cody/Research/clean_data/prepare.csv"
    output_path = r"C:/Cody/Research/temp_file"
elif select_mode == 1:
    file_path = r"D:/Research/clean_data/prepare.csv"
    output_path = r"D:/Research/temp_file"
else:
    print("YoU SelecT ThE WronG MoDe!")
    exit(1)

## 匯入函數庫

In [1]:
# import sys
# !{sys.executable} -m pip install tqdm
from datetime import datetime, timedelta
import pandas as pd
import tqdm
import matplotlib.pyplot as plt
import os 

## 資料格式化

In [5]:
source_data = pd.read_csv(file_path)

source_data["全時間格式進入時間"] = pd.to_datetime(source_data["全時間格式進入時間"])
source_data["全時間格式出場時間"] = pd.to_datetime(source_data["全時間格式出場時間"])
# 檢查數據
print("數據預覽:")
print(source_data.head())
print("缺失值檢查:")
print("資料數量:", len(source_data))
print("進入時間缺失:", source_data["全時間格式進入時間"].isna().sum())
print("出場時間缺失:", source_data["全時間格式出場時間"].isna().sum())

total_rows = len(source_data)
entry_missing_ratio = (source_data["全時間格式進入時間"].isna().sum() / total_rows) * 100
exit_missing_ratio = (source_data["全時間格式出場時間"].isna().sum() / total_rows) * 100
print(f"進入缺失資料占比: {entry_missing_ratio:.2f}%")
print(f"出場缺失資料占比: {exit_missing_ratio:.2f}%")

數據預覽:
        車號      票種      子場站 進出及付費狀態  校正狀態    進站設備    出站設備         進入日  \
0  KLC0802     臨停車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口內  2024/10/15   
1  AMG3703  學生計次汽車  光復校區停車場     已出站  比對一樣  南大門入口外  北大門出口內  2024/10/15   
2   1060MX   教職員汽車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口外  2024/10/15   
3   9826TU   教職員汽車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口外  2024/10/15   
4  KLN5097     臨停車  光復校區停車場     已出站  比對一樣  北大門入口外  北大門出口內  2024/10/15   

       進入時間         出場日      出場時間   停留時數 計價代碼                 紀錄時間  \
0  23:41:25  2024/10/15  23:44:42   0.05  NaN  2024-10-15 23:44:42   
1  23:40:38  2024/10/17  20:22:00  44.69  F11  2024-10-17 20:22:35   
2  23:39:43  2024/10/16  11:07:00  11.45  L12  2024-10-16 11:07:46   
3  23:39:19  2024/10/16  02:05:00   2.43  L12  2024-10-16 02:05:19   
4  23:34:17  2024/10/15  23:44:12   0.17  NaN  2024-10-15 23:44:12   

            全時間格式進入時間           全時間格式出場時間  
0 2024-10-15 23:41:25 2024-10-15 23:44:42  
1 2024-10-15 23:40:38 2024-10-17 20:22:00  
2 

## 建置紀錄表格

In [ ]:
# 先創建時間索引 30T 30min h 之類
time_index = pd.date_range(start="2024-01-01 00:00:00",end="2024-07-31 23:00:00",freq = "15min")

# 欄位對照
weekday_names = ['星期一', '星期二', '星期三', '星期四', '星期五', '星期六', '星期日']
columns = [f"{weekday_names}_{suffix}" for weekday in weekday_names for suffix in ['進','出','停留']]

# 初始化表格 第一個參數 data=0 表示所有 cell 都被預設為 0
# index 為索引
# columns 設定欄位名稱
record_table = pd.DataFrame(0,index=time_index,columns=columns)

skipped_count = 0      # 多少資料被略過
out_of_range_count = 0 # 是否有資料之紀錄超出範圍

for _,row in tqdm(source_data.iterrows(),total=len(source_data),desc="處理停車紀錄"):
    enter_time = row["全時間格式進入時間"]
    leave_time = row["全時間格是出場時間"]

    # 跳過空值
    if pd.isna(enter_time) or pd.isna(leave_time):
        skipped_count += 1
        continue

    # 跳過時間紀錄異常(出場早於或等於入場)
    if leave_time <= enter_time:
        skipped_count += 1
        tqdm.write(f"警告：車號 {row.get('車號', '未知')} 出場時間 {leave_time} 早於或等於進入時間 {enter_time}，跳過")
        continue

    # 超短時間進出(異常)
    if (leave_time - enter_time) < pd.Timedelta(seconds=60):
        skipped_count += 1
        tqdm.write(f"警告：車號 {row.get('車號', '未知')} 停留小於60秒，跳過")
        continue

    # 檢查時間範圍
    if enter_time < time_index[0] or leave_time > time_index[-1]:
        out_of_range_count += 1
        tqdm.write(f"警告：車號 {row.get('車號', '未知')} 時間超出範圍（{enter_time} 至 {leave_time}），跳過")
        continue

    '''
        統計進出場次數
    '''
    # 進場
    enter_point = enter_time.floor("15min")
    enter_weekday = weekday_names[enter_point.weekday()]
    # at 去快速存取或修改 dataframe 中單個 cell 的值 DataFrame.at[row_label, column_label]
    record_table.at[enter_point, f"{enter_point}_進"] += 1
    # 出場
    leave_point = enter_time.floor("15min")
    leave_weekday = weekday_names[leave_point.weekday()]
    record_table.at[leave_point, f"{leave_point}_出"] += 1

    '''*** 分時停留數量統計 ***'''
    # 停留時間未超過30分鐘不納入停留車統計
    # 我認為臨停族群非停留車探討之主要目標 (教職員、學生之長時停車情形多，以較長時停車作為主要目標)
    if leave_point - enter_point < pd.Timedelta(minutes=30):
        skipped_count += 1
        tqdm.write(f"車號 {row.get('車號', '未知')} 停留時間未超過30分鐘，跳過")
        continue

    '''
        假設：
            enter_point = 2024-01-01 13:10:00
            leave_point = 2024-01-01 13:40:00
            停留時間：13:40:00 - 13:10:00 = 30 分鐘
        結果（自動對齊到 15 分鐘間隔）：
            [2024-01-01 13:15:00, 2024-01-01 13:30:00]
            
        注意：13:10:00 會被對齊到 13:15:00，13:40:00 會被對齊到 13:30:00
    '''
    if (leave_point - enter_point) >= pd.Timedelta(minutes=15):  # 停留超過 15 分鐘計入該時段
        duration_periods = pd.date_range(start=enter_point, end=leave_point, freq='15min')
    else:
        duration_periods = pd.date_range(start=enter_point, end=leave_point - pd.Timedelta(minutes=15), freq='15min')

    for stay_time in duration_periods:
        if stay_time in record_table.index:
            stay_weekday = weekday_names[stay_time.weekday()]
            record_table.at[stay_time, f"{stay_weekday}_停留"] += 1 





